In [1]:
from architecture import ContextMatcher
import json
import torch
import torch.nn.functional as F
import numpy as np
import nltk
from tqdm.auto import tqdm
data_dir = '/hdd/unsupervised-summarization/'

In [2]:
vocab = json.load(open(data_dir+'data-giga/vocab.json', 'r'))

In [3]:
# from ELMo import getELMo
# elmo = getELMo(vocab, False)

In [4]:
# torch.save(elmo, data_dir+'elmo_model')

In [5]:
embeddings = np.load(data_dir+'data-giga/embeddings.npy')
embeddings = torch.from_numpy(embeddings).cuda()
elmo = torch.load(data_dir+'elmo_model')
elmo_uni = torch.load(data_dir+'elmo_model_uni')
lm_elmo = torch.load(data_dir+'LM-check')
with open(data_dir+'data-giga/lm_record.json') as f:
    lm_record = json.load(f)

/home/sean/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'ELMo.LanguageModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [6]:
from statisticalLM import StatisticalLM
lm = StatisticalLM(lm_record, vocab)

In [7]:
lm.forward(torch.LongTensor([[10, 213]]))

tensor([[0.0000e+00, 1.0494e-02, 1.0494e-02,  ..., 4.8521e-07, 5.1848e-07,
         4.2421e-07]])

In [8]:
matcher = ContextMatcher(embeddings, elmo, lm_elmo)
matcher.eval()
matcher_uni = ContextMatcher(embeddings, elmo_uni, lm_elmo)
matcher.eval()

ContextMatcher(
  (LM): LanguageModel(
    (embed): Embedding(50000, 1024)
    (lstm): LSTM(1024, 1024, num_layers=2, batch_first=True, dropout=0.5)
    (CE): CrossEntropyLoss()
  )
  (pretrained_elmo): Elmo(
    (_elmo_lstm): _ElmoBiLm(
      (_token_embedder): _ElmoCharacterEncoder(
        (char_conv_0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
        (char_conv_1): Conv1d(16, 32, kernel_size=(2,), stride=(1,))
        (char_conv_2): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
        (char_conv_3): Conv1d(16, 128, kernel_size=(4,), stride=(1,))
        (char_conv_4): Conv1d(16, 256, kernel_size=(5,), stride=(1,))
        (char_conv_5): Conv1d(16, 512, kernel_size=(6,), stride=(1,))
        (char_conv_6): Conv1d(16, 1024, kernel_size=(7,), stride=(1,))
        (_highways): Highway(
          (_layers): ModuleList(
            (0): Linear(in_features=2048, out_features=4096, bias=True)
            (1): Linear(in_features=2048, out_features=4096, bias=True)
          )
       

In [9]:
raw = "turnout was heavy for parliamentary elections monday in trinidad and tobage after a month of intensive campaigning throughout the country, one of the most prosperous in the caribbean."
# raw = "A dozen current and former staff from different areas of the State Department and at different stages of their careers who spoke to CNN said the Ukraine controversy has exacerbated divisions between political appointees and career diplomats, many of whom say the department is being politicized in ways that undermine US ties to other countries."

In [10]:
s = [list(nltk.word_tokenize(raw))]
# s = [['fuck' if i in ['the', 'The'] else i for i in sen] for sen in s]

In [11]:
print(s)
print(len(s[0]))

[['turnout', 'was', 'heavy', 'for', 'parliamentary', 'elections', 'monday', 'in', 'trinidad', 'and', 'tobage', 'after', 'a', 'month', 'of', 'intensive', 'campaigning', 'throughout', 'the', 'country', ',', 'one', 'of', 'the', 'most', 'prosperous', 'in', 'the', 'caribbean', '.']]
30


In [12]:
# s = [['I', 'have', 'a', 'dog', '!'], ['I', 'have', 'a', 'cat', '.']]

In [13]:
vocab_inv = {a:b for b,a in vocab.items()}

In [14]:
def similarity(x, y):
    return F.cosine_similarity(x, y, -1)

In [15]:
def count_score(source, now, target):
    max_score = -1
    max_index = 0

    for i in range(now+1, len(source)):
        score = similarity(source[i], target)
        if max_score < 0 or score > max_score:
            max_score = score
            max_index = i
            
    return max_index, max_score

In [16]:
def summary(sourse, cand, matcher, lm):
    output = []
    now = -1
    matcher.voronoi_split(torch.LongTensor(cand))
    
    while now < len(sourse)-1:

        index_list = []
        score_list = []
        
        dist = lm.forward(torch.LongTensor([output]))[0]
        t = 0
        for w in tqdm(cand):
            output_emb = matcher.embed(torch.LongTensor([output+[w]]))[0][-1] ## could be done in batch
            index, score = count_score(sourse, now, output_emb)
            index_list.append(index)
            fm = dist[matcher.get_cell_mates(w)].sum()
            t += fm
            score_list.append(score + fm)
#             print(vocab_inv[w], index, score, fm)

        score_list = torch.FloatTensor(score_list)
        qcm = F.softmax(score_list, -1)
        k = np.argmax(score_list)
#         print(k, score_list[k])

        output.append(cand[k])
        now = index_list[k]
#         print(vocab_inv[output[-1]], now)
    
    output = [vocab_inv[i] for i in output]
    
    return output

In [17]:
def summary_batch(s, matcher, lm):
    print(s)
    s_index = [[ vocab.get(w.lower(), vocab["<unk>"]) for w in sen] for sen in s]
    s_index = torch.LongTensor(s_index)
    s_emb = matcher.embed(s_index)
    output_list = []
    for index, target in enumerate(s_emb):
        vocab_list = matcher.candidate_list(s_index[index], 2).numpy()
        output = summary(target, vocab_list, matcher, lm)
        output_list.append(output)
    return output_list

In [18]:
result1 = summary_batch(s, matcher, lm)

[['turnout', 'was', 'heavy', 'for', 'parliamentary', 'elections', 'monday', 'in', 'trinidad', 'and', 'tobage', 'after', 'a', 'month', 'of', 'intensive', 'campaigning', 'throughout', 'the', 'country', ',', 'one', 'of', 'the', 'most', 'prosperous', 'in', 'the', 'caribbean', '.']]


In [19]:
result2 = summary_batch(s, matcher_uni, lm)

[['turnout', 'was', 'heavy', 'for', 'parliamentary', 'elections', 'monday', 'in', 'trinidad', 'and', 'tobage', 'after', 'a', 'month', 'of', 'intensive', 'campaigning', 'throughout', 'the', 'country', ',', 'one', 'of', 'the', 'most', 'prosperous', 'in', 'the', 'caribbean', '.']]


In [20]:
print([' '.join(s) for s in result1], [' '.join(s) for s in result2])
print(' '.join(s[0]))
print(len(s[0]))

['.'] ['turnout was campaigned .']
turnout was heavy for parliamentary elections monday in trinidad and tobage after a month of intensive campaigning throughout the country , one of the most prosperous in the caribbean .
30


In [21]:
s_index = [[ vocab.get(w.lower(), vocab["<unk>"]) for w in sen] for sen in s]
s_index = torch.LongTensor(s_index)

In [22]:
matcher.voronoi_split(matcher.candidate_list(s_index[0], 2))

In [ ]:
for i in range(len(s_index[0])):
    print(matcher_uni.embed(s_index[0][:i+1].unsqueeze(0))[:, -1, :].sum(), vocab_inv[s_index[0][i].item()])

In [ ]:
matcher.candidate_list(s_index[0], 4)

In [ ]:
matcher.candidate_map

In [ ]:
matcher.embeddings[vocab['turnout']]

In [ ]:
matcher.embed(s_index[0][:54].unsqueeze(0))[:, -1, :].sum()

In [ ]:
s_index[0][:54]

In [ ]:
embs = [] 
for i in range(10):
    e = matcher.embed(torch.LongTensor([[vocab['<S>'], vocab['chicken'], vocab['.']]]))[:,-1, :]
    embs.append(e.squeeze())
embs = torch.stack(embs, dim=0)
print(embs.shape)

In [ ]:
embs = F.normalize(embs, p=2, dim=-1)

In [ ]:
print(torch.matmul(embs, embs.transpose(-2, -1)))

In [ ]:
vocab['the']